### 1.0 MLP

In [1]:


import numpy as np
import tensorflow as tf

BATCH_SIZE = 32
INPUT_DIM = 51

def generate_data_v1(BATCH_SIZE=32, INPUT_DIM=128):
    # 生成两个输入特征
    inputs_0 = np.random.rand(BATCH_SIZE, INPUT_DIM).astype(np.float32)
    inputs_1 = np.random.rand(BATCH_SIZE, INPUT_DIM).astype(np.float32)

    # 生成伪标签：正样本的特征均值偏高
    feature_mean = (np.mean(inputs_0, axis=1) + np.mean(inputs_1, axis=1)) / 2

    # 生成标签：正样本的概率与均值相关
    # 通过 sigmoid 引入非线性关系，加入噪声使模型难以完美拟合
    prob = 1 / (1 + np.exp(-feature_mean + np.random.normal(0, 1.0, size=BATCH_SIZE)))
    
    # 将 prob 重塑为 (BATCH_SIZE, 1) 以匹配 size=(BATCH_SIZE, 1)
    prob = prob.reshape(BATCH_SIZE, 1)
    
    # 生成二项式分布的标签
    labels = np.random.binomial(1, prob, size=(BATCH_SIZE, 1)).astype(np.float32)

    # 构建 Dataset
    dataset = tf.data.Dataset.from_tensor_slices(
        ({'input_0': inputs_0, 'input_1': inputs_1}, labels)
    ).shuffle(1000).batch(BATCH_SIZE)

    return dataset

# 创建 Dataset
dataset = generate_data_v1(BATCH_SIZE, INPUT_DIM)


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import numpy as np
import tensorflow as tf

# 设置随机种子以保证每次生成相同的数据
np.random.seed(42)
tf.set_random_seed(42)


# === 模型定义 ===
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

def build_model(input_dim):
    model = Sequential(name="simple_mlp")
    model.add(Dense(64, activation='relu', input_shape=(input_dim,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    return model

# 超参数
BATCH_SIZE = 32
INPUT_DIM = 51
EPOCHS = 10
LR = 0.001



# 构建模型
model_input = tf.placeholder(tf.float32, shape=[None, 2 * INPUT_DIM], name='model_input')
labels_ph = tf.placeholder(tf.float32, shape=[None, 1], name='labels')

model = build_model(input_dim=2 * INPUT_DIM)
logits = model(model_input)
loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        labels=labels_ph, logits=logits
    )
)

optimizer = tf.train.AdamOptimizer(learning_rate=LR)
train_op = optimizer.minimize(loss)

# AUC 指标
predictions = tf.sigmoid(logits)
auc_metric, auc_update_op = tf.metrics.auc(
    labels=labels_ph,
    predictions=predictions,
    num_thresholds=200
)
init_local_vars = tf.local_variables_initializer()

# 启动会话
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(EPOCHS):
        # 每个 epoch 重新生成相同的数据集
#         dataset = generate_data_v1(BATCH_SIZE, INPUT_DIM)
        iterator = dataset.make_one_shot_iterator()
        next_batch = iterator.get_next()

        # 初始化 AUC 计算
        sess.run(init_local_vars)

        while True:
            try:
                inputs_batch, labels_batch = sess.run(next_batch)
                x = np.concatenate(
                    [inputs_batch['input_0'], inputs_batch['input_1']],
                    axis=1
                )

                _, loss_val, _ = sess.run(
                    [train_op, loss, auc_update_op],
                    feed_dict={
                        model_input: x,
                        labels_ph: labels_batch
                    }
                )
            except tf.errors.OutOfRangeError:
                break

        auc_result = sess.run(auc_metric)
        print("Epoch %d, Loss: %.4f, AUC: %.4f" % (epoch, loss_val, auc_result))

Epoch 0, Loss: 0.6877, AUC: 0.4722
Epoch 1, Loss: 0.6715, AUC: 0.5873
Epoch 2, Loss: 0.6567, AUC: 0.6964
Epoch 3, Loss: 0.6430, AUC: 0.7480
Epoch 4, Loss: 0.6298, AUC: 0.8353
Epoch 5, Loss: 0.6168, AUC: 0.8730
Epoch 6, Loss: 0.6051, AUC: 0.8869
Epoch 7, Loss: 0.5937, AUC: 0.9087
Epoch 8, Loss: 0.5826, AUC: 0.9127
Epoch 9, Loss: 0.5718, AUC: 0.9345


### 2.0 悟空

In [3]:
# -*- coding: utf-8 -*-
import numpy as np
import tensorflow as tf
from wukong import Wukong  # 假设 wukong.py 文件已包含模型定义
from tensorflow.python.keras.regularizers import l2
import logging
from IPython import display


# ========== 自定义日志处理器 ==========
class DisplayLogHandler(logging.Handler):
    def emit(self, record):
        msg = self.format(record)
        formatted_msg = "**{}**".format(msg)
        display.display(display.Markdown(formatted_msg))


# ========== 配置日志 ==========
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# 添加 DisplayLogHandler（用于 Jupyter）
handler1 = DisplayLogHandler()
formatter1 = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler1.setFormatter(formatter1)
logger.addHandler(handler1)

# 添加 StreamHandler（用于终端输出）
handler2 = logging.StreamHandler()
formatter2 = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler2.setFormatter(formatter2)
logger.addHandler(handler2)


# ========== 数据生成函数（返回 Dataset） ==========
def generate_data_v1(BATCH_SIZE=32, INPUT_DIM=128):
    inputs_0 = np.random.rand(BATCH_SIZE, INPUT_DIM).astype(np.float32)
    inputs_1 = np.random.rand(BATCH_SIZE, INPUT_DIM).astype(np.float32)
    feature_mean = (np.mean(inputs_0, axis=1) + np.mean(inputs_1, axis=1)) / 2
    prob = 1 / (1 + np.exp(-feature_mean + np.random.normal(0, 1.0, size=BATCH_SIZE)))
    prob = prob.reshape(BATCH_SIZE, 1)
    labels = np.random.binomial(1, prob, size=(BATCH_SIZE, 1)).astype(np.float32)

    # 创建 Dataset
    dataset = tf.data.Dataset.from_tensor_slices(
        ({'input_0': inputs_0, 'input_1': inputs_1}, labels)
    ).shuffle(1000).batch(BATCH_SIZE)

    return dataset


# ========== 模型参数 ==========
BATCH_SIZE = 32
INPUT_DIM = 128
EPOCHS = 10

dataset = generate_data_v1(BATCH_SIZE=BATCH_SIZE, INPUT_DIM=INPUT_DIM)


# ========== 损失函数和指标 ==========
def binary_crossentropy_loss(logits, labels):
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
    return tf.reduce_mean(loss)

def auc_metric(logits, labels):
    labels = tf.cast(labels, tf.bool)
    labels = tf.reshape(labels, [-1])
    logits_sigmoid = tf.sigmoid(logits)
    logits_sigmoid = tf.reshape(logits_sigmoid, [-1])

    # 获取 shape
    labels_shape = tf.shape(labels)
    logits_sigmoid_shape = tf.shape(logits_sigmoid)

    # 增加调试日志
    logging.debug("labels shape: %s, logits_sigmoid shape: %s" % (labels_shape, logits_sigmoid_shape))

    # 使用 tf.Assert 替代 Python assert
    with tf.control_dependencies([
        tf.Assert(tf.equal(labels_shape[0], logits_sigmoid_shape[0]), 
                  [tf.shape(labels), tf.shape(logits_sigmoid), 
                   "labels and logits_sigmoid must have the same shape"])
    ]):
        # 在 control_dependencies 上下文中执行 AUC 计算
        auc_value, auc_update = tf.metrics.auc(labels, logits_sigmoid)

    # 增加调试日志：记录 auc_update 和 auc_value 的具体值
    logging.debug("auc_update: %s, auc_value: %s" % (auc_update, auc_value))
    return auc_update, auc_value


# ========== 创建模型实例 ==========
model = Wukong(
    num_layers=2,
    num_emb_in=102,  # 51 * 2
    dim_emb=51,
    num_emb_lcb=16,
    num_emb_fmb=16,
    rank_fmb=8,
    num_hidden_wukong=2,
    dim_hidden_wukong=16,
    num_hidden_head=2,
    dim_hidden_head=32,
    dim_output=1,
    dropout=0.1,
#     kernel_regularizer=l2(1e-4)
    kernel_regularizer=l2(1e-3)
    
)


# ========== 构建图 ==========

input_0 = tf.placeholder(tf.float32, [BATCH_SIZE, INPUT_DIM])
input_1 = tf.placeholder(tf.float32, [BATCH_SIZE, INPUT_DIM])
labels = tf.placeholder(tf.float32, [BATCH_SIZE, 1])


inputs = tf.concat([input_0, input_1], axis=1)  # 合并为 (batch, 256)

logits = model(inputs)
loss = binary_crossentropy_loss(logits, labels)

optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(loss, var_list=model.trainable_weights)

auc_update, auc_value = auc_metric(logits, labels)


# ========== 初始化会话 ==========
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

with tf.Session() as sess:
    sess.run(init_op)

    for epoch in range(EPOCHS):
#         logging.debug("Epoch %d started" % epoch)
#         dataset = generate_data_v1(BATCH_SIZE=BATCH_SIZE, INPUT_DIM=INPUT_DIM)
        iterator = dataset.make_one_shot_iterator()
        next_batch = iterator.get_next()

        # 重置 AUC 局部变量
        sess.run(tf.local_variables_initializer())

        total_loss = 0.0
        step_count = 0

        while True:
            try:
                inputs_batch, labels_batch = sess.run(next_batch)

                # 增加调试日志
#                 logging.debug("inputs_batch['input_0'] shape:{}".format(inputs_batch['input_0'].shape))
#                 logging.debug("inputs_batch['input_1'] shape:{}".format(inputs_batch['input_1'].shape))
#                 logging.debug("labels_batch shape:{}".format(labels_batch.shape))

                # 同步运行 train_op、loss、auc_update
                _, loss_val, auc_update_val = sess.run(
                    [train_op, loss, auc_update],
                    feed_dict={
                        input_0: inputs_batch['input_0'],
                        input_1: inputs_batch['input_1'],
                        labels: labels_batch
                    }
                )

                # 增加调试日志：记录 auc_update_val 的值
#                 logging.debug("auc_update_val: %f" % auc_update_val)

                total_loss += loss_val
                step_count += 1

            except tf.errors.OutOfRangeError:
#                 logging.debug("End of dataset reached")
                break

        # 增加调试日志：确保 auc_value 被正确计算
        auc_result_val = sess.run(auc_value)
#         logging.debug("Final AUC result: %f" % auc_result_val)

        print("Epoch %d, Loss: %.4f, AUC: %.4f" % (epoch, total_loss / step_count, auc_result_val))


DEBUG:root:f01:inputs : Tensor("concat:0", shape=(32, 256), dtype=float32)


**2025-07-04 23:45:31,754 - DEBUG - f01:inputs : Tensor("concat:0", shape=(32, 256), dtype=float32)**

**2025-07-04 23:45:31,754 - DEBUG - f01:inputs : Tensor("concat:0", shape=(32, 256), dtype=float32)**

2025-07-04 23:45:31,754 - DEBUG - f01:inputs : Tensor("concat:0", shape=(32, 256), dtype=float32)
DEBUG:root:f02:outputs : Tensor("wukong/Reshape:0", shape=(32, 102, 51), dtype=float32)


**2025-07-04 23:45:31,791 - DEBUG - f02:outputs : Tensor("wukong/Reshape:0", shape=(32, 102, 51), dtype=float32)**

...

2025-07-04 23:45:34,020 - DEBUG - auc_update: Tensor("auc_1/update_op:0", shape=(), dtype=float32), auc_value: Tensor("auc_1/value:0", shape=(), dtype=float32)
DEBUG:tensorflow:PAI_ENABLE_ONLINE_TUNING 0


**2025-07-04 23:45:34,922 - DEBUG - PAI_ENABLE_ONLINE_TUNING 0**

**2025-07-04 23:45:34,922 - DEBUG - PAI_ENABLE_ONLINE_TUNING 0**

2025-07-04 23:45:34,922 - DEBUG - PAI_ENABLE_ONLINE_TUNING 0


Epoch 0, Loss: 0.6944, AUC: 0.4568
Epoch 1, Loss: 2.0042, AUC: 0.5000
Epoch 2, Loss: 1.2139, AUC: 0.9523
Epoch 3, Loss: 0.8257, AUC: 0.9568
Epoch 4, Loss: 0.6400, AUC: 0.9909
Epoch 5, Loss: 0.5928, AUC: 0.9955
Epoch 6, Loss: 0.5894, AUC: 1.0000
Epoch 7, Loss: 0.5919, AUC: 1.0000
Epoch 8, Loss: 0.5952, AUC: 1.0000
Epoch 9, Loss: 0.5964, AUC: 1.0000
